In [1]:
import h2o
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.18" 2023-01-17 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.18+9-LTS-195); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.18+9-LTS-195, mixed mode)
  Starting server from /Users/dodam/Desktop/Project/venvs/jupyterlab/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/18/gyvv8jl912s4m49p2kzkzznc0000gn/T/tmp12oo1m6f
  JVM stdout: /var/folders/18/gyvv8jl912s4m49p2kzkzznc0000gn/T/tmp12oo1m6f/h2o_dodam_started_from_python.out
  JVM stderr: /var/folders/18/gyvv8jl912s4m49p2kzkzznc0000gn/T/tmp12oo1m6f/h2o_dodam_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_dodam_7brhr7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
train = h2o.import_file('../data/BookRecommendations/train.csv')
test = h2o.import_file('../data/BookRecommendations/test.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
x = train.columns
y = "Book-Rating"
x.remove(y)

In [5]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(
    max_models=10,
    seed=42,
    max_runtime_secs=360,
    sort_metric='RMSE'

)

aml.train(
    x=x,
    y=y,
    training_frame=train
)

leaderboard = aml.leaderboard
print(leaderboard.head())

AutoML progress: |
23:04:17.78: AutoML: XGBoost is not available; skipping it.
23:04:17.103: _train param, Dropping bad and constant columns: [ID]

████████████
23:05:18.98: _train param, Dropping bad and constant columns: [ID]

█
23:05:29.400: _train param, Dropping bad and constant columns: [ID]

████████
23:06:14.775: _train param, Dropping bad and constant columns: [ID]

█
23:06:22.109: _train param, Dropping bad and constant columns: [ID]

██
23:06:29.305: _train param, Dropping bad and constant columns: [ID]

███████████████████████████████████████| (done) 100%
model_id                           rmse      mse      mae      rmsle    mean_residual_deviance
GBM_1_AutoML_1_20230419_230416  3.48461  12.1425  3.03241    1.05993                   12.1425
GBM_4_AutoML_1_20230419_230416  3.51542  12.3582  3.09414  nan                         12.3582
GBM_3_AutoML_1_20230419_230416  3.54663  12.5786  3.14325  nan                         12.5786
GBM_2_AutoML_1_20230419_230416  3.56379  12.70

In [6]:
model = aml.leader

In [7]:
pred = model.predict(test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [8]:
preds = pd.DataFrame(pred.as_data_frame())

In [9]:
submission = pd.read_csv('../data/BookRecommendations/sample_submission.csv')

In [10]:
submission['Book-Rating'] = preds['predict']

In [11]:
submission.to_csv('submission02.csv', index=False)